In [1]:
import pandas as pd

In [46]:
# load all our census data into dataframes
census_df = pd.read_csv('./csv/census_canada_scrape.csv')
doctor_count = pd.read_csv('./csv/doctor_city_count.csv')
closed_pharmacies_count = pd.read_csv('./csv/closed.csv')
open_pharmacies_count = pd.read_csv('./csv/combined.csv')

In [47]:
# Replace stoufville with more common name
census_df['City'] = census_df['City'].str.replace("WhitchurchStouffville", "Stouffville")

# combine the doctor count with the census dataframe
for index,city in doctor_count.iterrows():
    census_df.loc[census_df['City'] == city[0], 'Doctors'] = city[1] 

# ratio of persons per doctor
census_df['Population, 2021 / Doctors'] = census_df['Population, 2021'] / census_df['Doctors']

# combine the doctor count with the census dataframe
for index,city in closed_pharmacies_count.iterrows():
    census_df.loc[census_df['City'] == city[0], 'Closed Pharmacies'] = city[1] 
    
census_df['Population, 2021 / Closed Pharmacies'] = census_df['Population, 2021'] / census_df['Closed Pharmacies']

# combine the doctor count with the census dataframe
for index,city in open_pharmacies_count.iterrows():
    census_df.loc[census_df['City'] == city[0], 'Open Pharmacies'] = city[1] 
    
# Calculate the ratio of persons per pharmacy
census_df['Population, 2021 / Open Pharmacies'] = census_df['Population, 2021'] / census_df['Open Pharmacies']
census_df['Population, 2026 / Open Pharmacies'] = census_df['Population, 2026'] / census_df['Open Pharmacies']

# drop unnecessary columns
census_df.drop(columns=['Population, 2021','Population, 2026',  'Population, 2016', 'Unnamed: 0', ], inplace=True)
census_df.drop(index=[28, 46], inplace=True)

# fill in missing values
census_df['Closed Pharmacies'].fillna(0, inplace=True)
census_df['Population, 2021 / Closed Pharmacies'].fillna(999999, inplace=True)

In [63]:
positive_columns = census_df.columns[[ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,20, 21]]
negative_columns = census_df.columns[[18, 19]]

rank_df = pd.DataFrame()
rank_df['City'] = census_df['City']

for cols in positive_columns:
    rank_df[cols] = pd.qcut(census_df[cols], 10, labels=False, duplicates='drop') + 1
    
for cols in negative_columns:
    rank_df[cols] = 9 - pd.qcut(census_df[cols], 10, labels=False, duplicates='drop')

In [64]:
rank_df

,City,Total private dwellings,Population density per square kilometre,Median after-tax income in 2020 among recipients ($),Median after-tax income of household in 2020 ($),65 years and over,0 to 24 years,25 to 64 years,Median age of the population,Owner,...,"Population percentage change, 2016 to 2021",walk score,bike score,transit score,Doctors,"Population, 2021 / Doctors",Open Pharmacies,"Population, 2021 / Open Pharmacies",Closed Pharmacies,"Population, 2021 / Closed Pharmacies"
0,Cobourg,2,5,5,4,3,1,1,10,1,...,4,8,8,8,3.0,4.0,2,1,9,3
1,Port Hope,1,1,7,6,2,1,1,10,1,...,2,4,6,6,1.0,9.0,1,2,9,9
2,Belleville,4,3,3,2,5,4,5,8,5,...,7,5,7,7,6.0,3.0,5,3,9,7
3,Kingston,8,3,6,4,8,7,7,5,7,...,6,10,7,7,10.0,1.0,8,4,7,9
4,Peterborough,6,8,1,1,6,5,5,6,5,...,2,10,10,10,7.0,2.0,7,3,9,5
5,Lindsay,2,8,1,1,2,2,2,9,2,...,6,7,7,7,3.0,3.0,3,1,9,3
6,Sudbury,9,1,8,6,9,8,8,6,8,...,2,7,4,4,8.0,5.0,7,9,9,3
7,Ottawa,10,4,10,7,10,10,10,3,10,...,7,10,10,10,10.0,2.0,10,10,6,5
8,Orillia,3,7,2,1,3,2,3,9,3,...,6,8,5,5,6.0,1.0,5,1,9,3
9,Niagara Falls,6,4,1,3,7,6,6,7,6,...,6,3,3,3,4.0,8.0,7,3,9,5
